In [5]:
# -*- coding: utf-8 -*-
"""
請求書_YYYYMMDD.xlsx を新規作成（すべての修正反映版）

反映内容
- ファイル名: 請求書_YYYYMMDD.xlsx（実行日の現在日付）
- 日付表示: yyyy/mm/dd（例: 2026/02/08）
- 「請求書」タイトル: 通常フォント（他と同じ）
- 右上 No. 0001 / 日付: 塗りなし・罫線なし（文字だけ）
- 罫線: 全部なし（明細表・合計ブロック等も罫線なし）
- 表ヘッダ（商品名/数量/単価/金額）: 通常フォント・塗りつぶしなし
- 金額列（E列）に、明細(20,000 / 15,000)の直下へ合計金額(35,000)を追加（値のみ）
- 合計/消費税/税込合計の名称は商品名列（B列）に配置
- 「小計」表記は使わず「合計」に統一
"""

from datetime import date
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border


def create_invoice_xlsx():
    today = date.today()
    filename = f"請求書_{today.strftime('%Y%m%d')}.xlsx"

    wb = Workbook()
    ws = wb.active
    ws.title = "請求書"

    # -----------------------------
    # 列幅・行高
    # -----------------------------
    col_widths = {
        "A": 2.0,
        "B": 18.0,
        "C": 8.0,
        "D": 10.0,
        "E": 10.0,
        "F": 6.0,
        "G": 14.0,
    }
    for col, w in col_widths.items():
        ws.column_dimensions[col].width = w

    for r in range(1, 50):
        ws.row_dimensions[r].height = 18

    # -----------------------------
    # スタイル（罫線なし統一）
    # -----------------------------
    font_body = Font(size=11)
    align_left = Alignment(horizontal="left", vertical="center")
    align_center = Alignment(horizontal="center", vertical="center")
    align_right = Alignment(horizontal="right", vertical="center")

    border_none = Border()
    fill_none = PatternFill()

    # -----------------------------
    # タイトル（通常フォント）
    # -----------------------------
    ws["B2"].value = "請求書"
    ws["B2"].font = font_body
    ws["B2"].alignment = align_left
    ws["B2"].border = border_none

    # -----------------------------
    # 固定データ（会社情報）
    # -----------------------------
    fixed_lines = [
        "株式会社ABC",
        "〒101-0022 東京都千代田区神田練塀町300",
        "TEL:03-1234-5678 FAX:03-1234-5678",
        "担当者名:鈴木一郎 様",
    ]
    start_row = 4
    for i, text in enumerate(fixed_lines):
        c = ws[f"B{start_row + i}"]
        c.value = text
        c.font = font_body
        c.alignment = align_left
        c.border = border_none

    # -----------------------------
    # No. / 日付（文字のみ：塗りなし・罫線なし）
    # -----------------------------
    ws["F4"].value = "No."
    ws["F4"].font = font_body
    ws["F4"].alignment = align_left
    ws["F4"].border = border_none

    ws["G4"].value = "0001"
    ws["G4"].font = font_body
    ws["G4"].alignment = align_center
    ws["G4"].fill = fill_none
    ws["G4"].border = border_none

    ws["F5"].value = "日付"
    ws["F5"].font = font_body
    ws["F5"].alignment = align_left
    ws["F5"].border = border_none

    ws["G5"].value = today
    ws["G5"].number_format = "yyyy/mm/dd"  # 例: 2026/02/08
    ws["G5"].font = font_body
    ws["G5"].alignment = align_center
    ws["G5"].fill = fill_none
    ws["G5"].border = border_none

    # -----------------------------
    # 明細ヘッダ（通常フォント・塗りなし・罫線なし）
    # -----------------------------
    headers = ["商品名", "数量", "単価", "金額"]
    for col, text in zip(["B", "C", "D", "E"], headers):
        c = ws[f"{col}10"]
        c.value = text
        c.font = font_body
        c.alignment = align_center
        c.fill = fill_none
        c.border = border_none

    # -----------------------------
    # 明細データ
    # -----------------------------
    items = [
        ("商品A", 2, 10000, 20000),
        ("商品B", 1, 15000, 15000),
    ]

    data_start = 11
    for i, (name, qty, unit, amount) in enumerate(items):
        r = data_start + i

        ws[f"B{r}"].value = name
        ws[f"B{r}"].font = font_body
        ws[f"B{r}"].alignment = align_left
        ws[f"B{r}"].border = border_none

        ws[f"C{r}"].value = qty
        ws[f"C{r}"].font = font_body
        ws[f"C{r}"].alignment = align_center
        ws[f"C{r}"].border = border_none

        ws[f"D{r}"].value = unit
        ws[f"D{r}"].font = font_body
        ws[f"D{r}"].alignment = align_right
        ws[f"D{r}"].number_format = "#,##0"
        ws[f"D{r}"].border = border_none

        ws[f"E{r}"].value = amount
        ws[f"E{r}"].font = font_body
        ws[f"E{r}"].alignment = align_right
        ws[f"E{r}"].number_format = "#,##0"
        ws[f"E{r}"].border = border_none

    # -----------------------------
    # 金額列の直下に合計金額（値のみ）
    # -----------------------------
    last_item_row = data_start + len(items) - 1
    amount_total_row = last_item_row + 1

    subtotal = sum(x[3] for x in items)

    ws[f"E{amount_total_row}"].value = subtotal
    ws[f"E{amount_total_row}"].font = font_body
    ws[f"E{amount_total_row}"].alignment = align_right
    ws[f"E{amount_total_row}"].number_format = "#,##0"
    ws[f"E{amount_total_row}"].border = border_none

    # -----------------------------
    # 合計 / 消費税 / 税込合計（ラベルはB列）
    # -----------------------------
    tax = int(subtotal * 0.10)  # 10%想定
    total = subtotal + tax

    summary_rows = [
        ("合計", subtotal),
        ("消費税", tax),
        ("税込合計", total),
    ]

    summary_start = amount_total_row + 2  # 1行空けて開始
    for i, (label, val) in enumerate(summary_rows):
        r = summary_start + i

        ws[f"B{r}"].value = label
        ws[f"B{r}"].font = font_body
        ws[f"B{r}"].alignment = align_left
        ws[f"B{r}"].border = border_none

        ws[f"E{r}"].value = val
        ws[f"E{r}"].font = font_body
        ws[f"E{r}"].alignment = align_right
        ws[f"E{r}"].number_format = "#,##0"
        ws[f"E{r}"].border = border_none

    wb.save(filename)
    return filename


if __name__ == "__main__":
    print(create_invoice_xlsx())


請求書_20260208.xlsx
